<a href="https://colab.research.google.com/github/Danielmzm081199/IntroalaIA/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Integrantes del equipo:
          # Daniel Munoz Mejia  cc: 1020494553 --->  Ingenieria de Sistemas Virtual
          # Cristian Florez Calderon cc: 1035233651 ---> Bioingenieria
          # William Daniel cuaical cc: 1007292685 ---> Ingenieria Industrial

ENTREGA 2

además del contenido de la entrega 1 en github:

Video subido a YouTube y enlazado en README.md

Elabora un vídeo de entre 3mins y 4mins.

En los primeros 20 segs del vídeo todos los miembros del equipo han de presentarse con la cámara, enunciando su nombre y el programa en el que están matriculados.

En el video ha de explicar brevemente el estado de avance del proyecto, dificultades actuales, etc.

Todos los miembros del equipo han de participar en la explicación.

README.md con los nombres, cédulas y progama en el que está inscrito cada participante y con el enlace la video en YouTube de esta entrega.

02 - preprocesado.ipynb donde se muestra cómo cargas el archivo train.csv de la competencia y realizas las operaciones de limpieza y preprocesado de datos que necesites (p.ej. convertir columnas en one-hot, normalizar valores de rangos, imputar datos faltantes, etc.)

# Cargar Librerias:

In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as inline
import re

## Cargar DataFrame desde Kaggle

In [ ]:
os.environ["KAGGLE_CONFIG_DIR"] = "."
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip
df = pd.read_csv("train.csv")
df.head()

 33% 7.00M/21.2M [00:00<00:00, 71.7MB/s]
100% 21.2M/21.2M [00:00<00:00, 123MB/s] 
Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [ ]:
print(df.columns)

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


#*Eliminamos la columna repetida*

In [ ]:
col_rep = df['FAMI_TIENEINTERNET'].equals(df['FAMI_TIENEINTERNET.1']) #Valida la columna repetida
print("Las columnas son iguales?: ",col_rep)

del df['FAMI_TIENEINTERNET.1'] #Se elimina la columna repetida

print(df.columns)#Se verifica que la columna repetida si fue eliminada

Las columnas son iguales?:  True
Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL'],
      dtype='object')


#Se comprueba si hay columnas con valores faltantes

In [ ]:
k = df.isna().sum() #Revisa si hay valores faltantes en cada columna, y si los hay se suma cuantas veces aparece un valor faltante (Na) por cada columna en el dataframe
k[k!=0] #se muestran solo las que no son 0

ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
dtype: int64

##Se eliminan columnas con valores faltantes que se deduce no son de mucha importancia para el modelo

In [ ]:
del df['FAMI_TIENELAVADORA']
del df['FAMI_TIENEAUTOMOVIL']
del df['ESTU_PAGOMATRICULAPROPIO']
print(df.columns)


Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')


#*Comprueba valores faltantes*

In [ ]:
k = df.isna().sum() #Se suma cuantas veces aparece un valor faltante (Na) por cada columna en el dataframe
k[k!=0] #se muestran solo las que no son 0

ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENECOMPUTADOR              38103
FAMI_EDUCACIONMADRE               23664
dtype: int64

Para hacer un mejor proceso para rellenar las columnas con datos que tienen combinación de texto y numeros, procedemos a manipular los datos de forma que en lugar de dejar el intervalo, usamos los valores numericos para calcular el valor medio de los mismos.

In [ ]:
df['ESTU_HORASSEMANATRABAJA'].value_counts() #se verifica el tipo de datos que hay en la columna

ESTU_HORASSEMANATRABAJA
Más de 30 horas        249352
0                      116550
Entre 11 y 20 horas    115857
Entre 21 y 30 horas     92693
Menos de 10 horas       87191
Name: count, dtype: int64

In [ ]:
def obtener_valor_medio(rango):
    if isinstance(rango, str):  #Verifica si el argumento dado es una cadena de texto(string)
        numeros = [int(num) for num in rango.split() if num.isdigit()] #Divide la cadena rango en palabras y crea una lista de números enteros (int) a partir de cada palabra que sea un número filtrando con if num.isdigit() filtra solo las palabras que son números.
        return sum(numeros) / len(numeros) #Devuelve el promedio de los números
    else:
        return float('nan')  #Devuelve un valor nan en caso de que no sea una cadena de texto

df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].apply(obtener_valor_medio) #se aplica la funcion a la columna del dataframe
df.head(12)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,10.0,Estrato 3,Si,Técnica o tecnológica incompleta,N,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0.0,Estrato 3,No,Técnica o tecnológica completa,N,Si,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,30.0,Estrato 3,Si,Secundaria (Bachillerato) completa,N,No,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0.0,Estrato 4,Si,No sabe,N,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,25.5,Estrato 3,Si,Primaria completa,N,Si,Primaria completa,medio-bajo
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,10.0,Estrato 5,Si,Educación profesional completa,N,Si,Secundaria (Bachillerato) completa,medio-alto
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,25.5,Estrato 2,Si,Educación profesional incompleta,N,Si,Técnica o tecnológica completa,alto
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,15.5,Estrato 2,Si,Primaria incompleta,N,No,Secundaria (Bachillerato) incompleta,medio-bajo
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,10.0,Estrato 1,Si,Secundaria (Bachillerato) completa,N,Si,Educación profesional incompleta,medio-bajo
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,30.0,Estrato 5,Si,Postgrado,N,Si,Postgrado,alto


Ahora si procedemos a remplazar los NA, para esto los rellenamos con la mediana de la columna


In [ ]:
df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(df['ESTU_HORASSEMANATRABAJA'].median()) #se rellenan los datos faltantes con la media de la columna
print(df['ESTU_HORASSEMANATRABAJA'].head(12))


0     10.0
1      0.0
2     30.0
3      0.0
4     25.5
5     10.0
6     25.5
7     15.5
8     10.0
9     30.0
10    15.5
11    30.0
Name: ESTU_HORASSEMANATRABAJA, dtype: float64


Ahora nos enfocamos en la variable: 'ESTU_VALORMATRICULAUNIVERSIDAD', la cual tambien nos muestra un rango con valores en millones, tomamos los valores numericos y dejamos el valor medio de estos.

In [ ]:
df['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts() #Muestra el conteo de la frecuencia para los valores unicos de esta columna

ESTU_VALORMATRICULAUNIVERSIDAD
Entre 1 millón y menos de 2.5 millones      204048
Entre 2.5 millones y menos de 4 millones    127430
Menos de 500 mil                             80263
Entre 500 mil y menos de 1 millón            78704
Entre 4 millones y menos de 5.5 millones     69736
Más de 7 millones                            68014
Entre 5.5 millones y menos de 7 millones     38490
No pagó matrícula                            19528
Name: count, dtype: int64

vemos que existe una caracteristica  y es que el: 'no pagó matricula' se repite cierta cantidad de veces, por lo cual vamos a considerar este valor como un cero.
Ademas se observa que existen valores que no tienen  la misma unidad, es conveniente utulizar una sola unidad (millones)



In [ ]:
def obtener_valor_medio2(rango):
    if isinstance(rango, str): #Verifica si el argumento dado es una cadena de texto(string)
        # Reemplazar datos específicas por los valores correspondientes
        rango = rango.replace("No pagó matrícula", "0")
        rango = rango.replace("Menos de 500 mil", "0.5")
        rango = rango.replace("Entre 500 mil y menos de 1 millón", "0.75")

        # Utilizar expresiones regulares para encontrar números decimales en la cadena
        numeros = re.findall(r"\d+\.\d+|\d+", rango) #Utiliza expresiones regulares \d+\.\d+|\d+ para buscar patrones de números con o sin decimales
        # Convertir los números a flotantes y calcular la media
        numeros = [float(num) for num in numeros] #Convierte los números encontrados a números flotantes (float).
        if numeros:
            return sum(numeros) / len(numeros) #Si en la lista hay numeros, devuelve el promedio de los números en la lista
        else:
            return float('nan') #Devuelve un valor nan en caso de que no sea un número
    else:
        return float('nan') #Devuelve un valor nan en caso de que no sea una cadena de texto


In [ ]:
df["ESTU_VALORMATRICULAUNIVERSIDAD"] = df["ESTU_VALORMATRICULAUNIVERSIDAD"].apply(obtener_valor_medio2) #se aplica la funcion a la columna del dataframe

df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(df['ESTU_VALORMATRICULAUNIVERSIDAD'].median()) #se rellenan los datos faltantes con la media de la columna
print(df['ESTU_VALORMATRICULAUNIVERSIDAD'].head(20))


0     6.25
1     3.25
2     3.25
3     4.75
4     3.25
5     7.00
6     3.25
7     1.75
8     6.25
9     3.25
10    1.75
11    0.75
12    3.25
13    0.50
14    0.75
15    4.75
16    4.75
17    1.75
18    0.50
19    1.75
Name: ESTU_VALORMATRICULAUNIVERSIDAD, dtype: float64


In [ ]:
k = df.isna().sum() #Se suma cuantas veces aparece un valor faltante (Na) por cada columna en el dataframe
k[k!=0] #se muestran solo las que no son 0

FAMI_ESTRATOVIVIENDA    32137
FAMI_TIENEINTERNET      26629
FAMI_EDUCACIONPADRE     23178
FAMI_TIENECOMPUTADOR    38103
FAMI_EDUCACIONMADRE     23664
dtype: int64

Seguimos con la columna FAMI_ESTRATOVIVIENDA que igualmente tiene texto y números

In [ ]:
print(df['FAMI_ESTRATOVIVIENDA'].value_counts()) #Muestra el conteo de la frecuencia para los valores unicos de esta columna

FAMI_ESTRATOVIVIENDA
Estrato 2      232671
Estrato 3      210685
Estrato 1      111991
Estrato 4       65514
Estrato 5       23608
Estrato 6       12605
Sin Estrato      3289
Name: count, dtype: int64


In [ ]:
df["FAMI_ESTRATOVIVIENDA"] = df["FAMI_ESTRATOVIVIENDA"].apply(obtener_valor_medio2) #se aplica la funcion a la columna del dataframe

df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna(df['FAMI_ESTRATOVIVIENDA'].median()) #se rellenan los datos faltantes con la media de la columna
print(df['FAMI_ESTRATOVIVIENDA'].head(20))

0     3.0
1     3.0
2     3.0
3     4.0
4     3.0
5     5.0
6     2.0
7     2.0
8     1.0
9     5.0
10    1.0
11    2.0
12    3.0
13    1.0
14    3.0
15    3.0
16    1.0
17    2.0
18    1.0
19    2.0
Name: FAMI_ESTRATOVIVIENDA, dtype: float64


In [ ]:
k = df.isna().sum() #Se suma cuantas veces aparece un valor faltante (Na) por cada columna en el dataframe
k[k!=0] #se muestran solo las que no son 0

FAMI_TIENEINTERNET      26629
FAMI_EDUCACIONPADRE     23178
FAMI_TIENECOMPUTADOR    38103
FAMI_EDUCACIONMADRE     23664
dtype: int64

##Se usa metodo OneHot para el resto de columnas con datos faltantes

In [ ]:
def one_hot_encode_column(df, column_name):
    df = df.fillna(method='ffill') #Este método rellena los valores faltantes en la columna, utilizando el último valor válido hacia adelante para rellenar los valores faltantes.

    unique_values = df[column_name].unique() # Se obtenien valores únicos en la columna

    # Se crean columnas one-hot para cada valor único
    for value in unique_values:
        df[column_name + '_' + str(value)] = (df[column_name] == value).astype(int)

    del df[column_name] # Se elimina la columna original

    return df

In [ ]:
df = one_hot_encode_column(df, 'FAMI_TIENEINTERNET') #se aplica la funcion a la columna del dataframe

df.head(20)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,FAMI_TIENEINTERNET_Si,FAMI_TIENEINTERNET_No
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,10.0,3.0,Técnica o tecnológica incompleta,N,Si,Postgrado,medio-alto,1,0
1,645256,20212,DERECHO,ATLANTICO,3.25,0.0,3.0,Técnica o tecnológica completa,N,Si,Técnica o tecnológica incompleta,bajo,0,1
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,30.0,3.0,Secundaria (Bachillerato) completa,N,No,Secundaria (Bachillerato) completa,bajo,1,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4.0,No sabe,N,Si,Secundaria (Bachillerato) completa,alto,1,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.5,3.0,Primaria completa,N,Si,Primaria completa,medio-bajo,1,0
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.00,10.0,5.0,Educación profesional completa,N,Si,Secundaria (Bachillerato) completa,medio-alto,1,0
6,47159,20183,INGENIERIA MECANICA,HUILA,3.25,25.5,2.0,Educación profesional incompleta,N,Si,Técnica o tecnológica completa,alto,1,0
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,15.5,2.0,Primaria incompleta,N,No,Secundaria (Bachillerato) incompleta,medio-bajo,1,0
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.25,10.0,1.0,Secundaria (Bachillerato) completa,N,Si,Educación profesional incompleta,medio-bajo,1,0
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,3.25,30.0,5.0,Postgrado,N,Si,Postgrado,alto,1,0


OneHot para Educacion_Padre

In [ ]:
df = one_hot_encode_column(df, 'FAMI_EDUCACIONPADRE') #se aplica la funcion a la columna del dataframe

df.head(20)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,ESTU_PRIVADO_LIBERTAD,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,...,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,10.0,3.0,N,Si,Postgrado,...,0,0,0,0,0,0,0,0,0,0
1,645256,20212,DERECHO,ATLANTICO,3.25,0.0,3.0,N,Si,Técnica o tecnológica incompleta,...,0,0,0,0,0,0,0,0,0,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,30.0,3.0,N,No,Secundaria (Bachillerato) completa,...,1,0,0,0,0,0,0,0,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4.0,N,Si,Secundaria (Bachillerato) completa,...,0,1,0,0,0,0,0,0,0,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.5,3.0,N,Si,Primaria completa,...,0,0,1,0,0,0,0,0,0,0
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.00,10.0,5.0,N,Si,Secundaria (Bachillerato) completa,...,0,0,0,1,0,0,0,0,0,0
6,47159,20183,INGENIERIA MECANICA,HUILA,3.25,25.5,2.0,N,Si,Técnica o tecnológica completa,...,0,0,0,0,1,0,0,0,0,0
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,15.5,2.0,N,No,Secundaria (Bachillerato) incompleta,...,0,0,0,0,0,1,0,0,0,0
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.25,10.0,1.0,N,Si,Educación profesional incompleta,...,1,0,0,0,0,0,0,0,0,0
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,3.25,30.0,5.0,N,Si,Postgrado,...,0,0,0,0,0,0,1,0,0,0


OneHot para TieneComputador

In [ ]:
df = one_hot_encode_column(df, 'FAMI_TIENECOMPUTADOR') #se aplica la funcion a la columna del dataframe
df.head(20)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,ESTU_PRIVADO_LIBERTAD,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,...,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Educación profesional completa,FAMI_EDUCACIONPADRE_Educación profesional incompleta,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_TIENECOMPUTADOR_Si,FAMI_TIENECOMPUTADOR_No
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,10.0,3.0,N,Postgrado,medio-alto,...,0,0,0,0,0,0,0,0,1,0
1,645256,20212,DERECHO,ATLANTICO,3.25,0.0,3.0,N,Técnica o tecnológica incompleta,bajo,...,0,0,0,0,0,0,0,0,1,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,30.0,3.0,N,Secundaria (Bachillerato) completa,bajo,...,0,0,0,0,0,0,0,0,0,1
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4.0,N,Secundaria (Bachillerato) completa,alto,...,0,0,0,0,0,0,0,0,1,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.5,3.0,N,Primaria completa,medio-bajo,...,1,0,0,0,0,0,0,0,1,0
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.00,10.0,5.0,N,Secundaria (Bachillerato) completa,medio-alto,...,0,1,0,0,0,0,0,0,1,0
6,47159,20183,INGENIERIA MECANICA,HUILA,3.25,25.5,2.0,N,Técnica o tecnológica completa,alto,...,0,0,1,0,0,0,0,0,1,0
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,15.5,2.0,N,Secundaria (Bachillerato) incompleta,medio-bajo,...,0,0,0,1,0,0,0,0,0,1
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.25,10.0,1.0,N,Educación profesional incompleta,medio-bajo,...,0,0,0,0,0,0,0,0,1,0
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,3.25,30.0,5.0,N,Postgrado,alto,...,0,0,0,0,1,0,0,0,1,0


OneHot para EducacionMadre

In [ ]:
df = one_hot_encode_column(df, 'FAMI_EDUCACIONMADRE') #se aplica la funcion a la columna del dataframe
df.head(20)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,ESTU_PRIVADO_LIBERTAD,RENDIMIENTO_GLOBAL,FAMI_TIENEINTERNET_Si,...,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,10.0,3.0,N,medio-alto,1,...,0,0,0,0,0,0,0,0,0,0
1,645256,20212,DERECHO,ATLANTICO,3.25,0.0,3.0,N,bajo,0,...,0,0,0,0,0,0,0,0,0,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,30.0,3.0,N,bajo,1,...,1,0,0,0,0,0,0,0,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4.0,N,alto,1,...,1,0,0,0,0,0,0,0,0,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.5,3.0,N,medio-bajo,1,...,0,1,0,0,0,0,0,0,0,0
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.00,10.0,5.0,N,medio-alto,1,...,1,0,0,0,0,0,0,0,0,0
6,47159,20183,INGENIERIA MECANICA,HUILA,3.25,25.5,2.0,N,alto,1,...,0,0,1,0,0,0,0,0,0,0
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,15.5,2.0,N,medio-bajo,1,...,0,0,0,1,0,0,0,0,0,0
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.25,10.0,1.0,N,medio-bajo,1,...,0,0,0,0,1,0,0,0,0,0
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,3.25,30.0,5.0,N,alto,1,...,0,0,0,0,0,0,0,0,0,0


Valores unicos para la variable de ESTU_PRIVADO_LIBERTAD

In [ ]:
print(df['ESTU_PRIVADO_LIBERTAD'].value_counts()) #Muestra el conteo de la frecuencia para los valores unicos de esta columna

ESTU_PRIVADO_LIBERTAD
N    692466
S        34
Name: count, dtype: int64


In [ ]:
df['ESTU_PRIVADO_LIBERTAD'] = df['ESTU_PRIVADO_LIBERTAD'].replace({'N': 'No', 'S': 'Si'}) #Se reemplaza los datos para que sean iguales a los usados anteriormente

df = one_hot_encode_column(df, 'ESTU_PRIVADO_LIBERTAD') #se aplica la funcion a la columna del dataframe

df.head(20)



,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,RENDIMIENTO_GLOBAL,FAMI_TIENEINTERNET_Si,FAMI_TIENEINTERNET_No,...,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,ESTU_PRIVADO_LIBERTAD_No,ESTU_PRIVADO_LIBERTAD_Si
0,904256,20212,ENFERMERIA,BOGOTÁ,6.25,10.0,3.0,medio-alto,1,0,...,0,0,0,0,0,0,0,0,1,0
1,645256,20212,DERECHO,ATLANTICO,3.25,0.0,3.0,bajo,0,1,...,0,0,0,0,0,0,0,0,1,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3.25,30.0,3.0,bajo,1,0,...,0,0,0,0,0,0,0,0,1,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4.75,0.0,4.0,alto,1,0,...,0,0,0,0,0,0,0,0,1,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3.25,25.5,3.0,medio-bajo,1,0,...,0,0,0,0,0,0,0,0,1,0
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,7.00,10.0,5.0,medio-alto,1,0,...,0,0,0,0,0,0,0,0,1,0
6,47159,20183,INGENIERIA MECANICA,HUILA,3.25,25.5,2.0,alto,1,0,...,1,0,0,0,0,0,0,0,1,0
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1.75,15.5,2.0,medio-bajo,1,0,...,0,1,0,0,0,0,0,0,1,0
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,6.25,10.0,1.0,medio-bajo,1,0,...,0,0,1,0,0,0,0,0,1,0
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,3.25,30.0,5.0,alto,1,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
print(df.columns) #Observamos cuantas columnas quedaron al final

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'RENDIMIENTO_GLOBAL', 'FAMI_TIENEINTERNET_Si',
       'FAMI_TIENEINTERNET_No',
       'FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta',
       'FAMI_EDUCACIONPADRE_Técnica o tecnológica completa',
       'FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa',
       'FAMI_EDUCACIONPADRE_No sabe', 'FAMI_EDUCACIONPADRE_Primaria completa',
       'FAMI_EDUCACIONPADRE_Educación profesional completa',
       'FAMI_EDUCACIONPADRE_Educación profesional incompleta',
       'FAMI_EDUCACIONPADRE_Primaria incompleta',
       'FAMI_EDUCACIONPADRE_Postgrado',
       'FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta',
       'FAMI_EDUCACIONPADRE_Ninguno', 'FAMI_EDUCACIONPADRE_No Aplica',
       'FAMI_TIENECOMPUTADOR_Si', 'FAMI_TIENECOMPUTADOR_No',
       'FAMI_EDUCACIONMADRE_Postgrado',
       'FAMI_EDUCACIONMADRE_

#*Reto*

In [ ]:
# Ordenar la columna alfabéticamente
df = df.sort_values('ESTU_PRGM_ACADEMICO') #Se ordenan los datos de la columna en orden alfabetico
df['ESTU_PRGM_ACADEMICO'].unique() #revisamos los datos unicos que hay en la columna

array(['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES',
       'ACTIVIDAD FISICA Y DEPORTE', 'ACUICULTURA', 'ADMINISTRACION',
       'ADMINISTRACION  FINANCIERA', 'ADMINISTRACION & SERVICIO',
       'ADMINISTRACION AERONAUTICA', 'ADMINISTRACION AGROPECUARIA',
       'ADMINISTRACION AMBIENTAL',
       'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES',
       'ADMINISTRACION BANCARIA Y FINANCIERA', 'ADMINISTRACION COMERCIAL',
       'ADMINISTRACION COMERCIAL Y DE MERCADEO',
       'ADMINISTRACION DE COMERCIO EXTERIOR',
       'ADMINISTRACION DE EMPRESAS',
       'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL',
       'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES',
       'ADMINISTRACION DE EMPRESAS AGROPECUARIAS',
       'ADMINISTRACION DE EMPRESAS COMERCIALES',
       'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES',
       'ADMINISTRACION DE EMPRESAS TURISTICA',
       'ADMINISTRACION DE EMPRESAS TURISTICAS',
       'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS',
       

In [ ]:
#Se halló una dificultad con los datos de esta columna ya que habian nombres que estaban repetidos pero con tildes o simbolos, por lo que los toma como duplicados
#haciendo que el tratamiento de esta columna sea complicado ya que son demasiados datos, se intento clasificar los datos de manera manual pero dado
#a la cantidad de datos no se pudo completar

In [ ]:
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('TEOLOGÍA', 'TEOLOGIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('SOCIOLOGÍA', 'SOCIOLOGIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('QU¿MICA FARMAC¿UTICA', 'QUÍMICA FARMACÉUTICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('QUÍMICA', 'QUIMICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PSICOLOGÍA', 'PSICOLOGIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PSICÓLOGA', 'PSICOLOGIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN GASTRONOM¿A', 'PROFESIONAL EN GASTRONOMIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN GASTRONOMÍA', 'PROFESIONAL EN GASTRONOMIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL  EN DISEÑO GRAFICO', 'PROFESIONAL EN DISEÑO GRAFICO')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('OPTOMETRÍA', 'OPTOMETRIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('ODONTOLOGÍA', 'ODONTOLOGIA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('NUTRICIÓN Y DIETÉTICA', 'NUTRICION Y DIETETICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('NUTRICIÓN Y DIETETICA', 'NUTRICION Y DIETETICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('MATEMÁTICA APLICADA', 'MATEMATICAS APLICADAS')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('MATEMÁTICAS', 'MATEMATICAS')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('MAESTRO EN MÚSICA', 'MAESTRO EN MUSICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA M¿SICA', 'LICENCIATURA MUSICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN TEOLOGÍA', 'LICENCIATURA EN TEOLOGIA')

In [ ]:
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN TECNOLOGÍA E INFORMÁTICA', 'LICENCIATURA EN TECNOLOGIA E INFORMATICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN PEDAGOGÍA INFANTIL', 'LICENCIATURA EN PEDAGOGIA INFANTIL')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN PEDAGOG¿A INFANTIL', 'LICENCIATURA EN PEDAGOGIA INFANTIL')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN MUSICA', 'LICENCIATURA EN MUSICA')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN MATEMÁTICAS', 'LICENCIATURA EN MATEMATICAS')
df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace('LICENCIATURA EN MATEM¿TICAS', 'LICENCIATURA EN MATEMATICAS')